<a href="https://colab.research.google.com/github/JulianCarax01/Fondamenti-web-app/blob/main/DijkstraSerialCPPMerge_prova_raff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ncu --mode launch-and-attach -o profile --target-processes all --nvtx --call-stack --section ComputeWorkloadAnalysis --section MemoryWorkloadAnalysis -f ./dijkstra_serialMerge.o


==ERROR== './dijkstra_serialMerge.o' does not exist or is not an executable. Please make sure to specify the absolute path to './dijkstra_serialMerge.o' if the executable is not in the local directory.


In [ ]:
%%writefile dijkstra_serialMerge_3kern.cu
#include <iostream>
#include <vector>
#include <cstdlib>
#include <climits>
#include <cuda.h>
#include <curand_kernel.h>
#include <chrono>

// Funzione per eseguire Dijkstra in modo seriale sulla CPU
void dijkstraSerial(const int *graph, int n, int start, int *distances) {
    // Array per tenere traccia dei nodi visitati
    std::vector<int> visited(n, 0);

    // Inizializza le distanze
    for (int i = 0; i < n; ++i) {
        distances[i] = (i == start) ? 0 : INT_MAX;
    }

    for (int i = 0; i < n; ++i) {
        int minDist = INT_MAX;
        int currentNode = -1;

        // Trova il nodo con la distanza minima non ancora visitato
        for (int j = 0; j < n; ++j) {
            if (!visited[j] && distances[j] < minDist) {
                minDist = distances[j];
                currentNode = j;
            }
        }

        if (currentNode == -1) break; // Nessun altro nodo raggiungibile

        // Marca il nodo corrente come visitato
        visited[currentNode] = 1;

        // Aggiorna le distanze per i nodi adiacenti
        for (int j = 0; j < n; ++j) {
            int weight = graph[currentNode * n + j]; // Peso dell'arco currentNode -> j
            if (weight > 0 && weight != INT_MAX) {  // Esiste un arco valido
                // Verifica per evitare overflow: dist[currentNode] + weight
                if (distances[currentNode] != INT_MAX &&
                    distances[currentNode] + weight < distances[j]) {
                    distances[j] = distances[currentNode] + weight;
                }
            }
        }
    }
}

// Kernel CUDA per aggiornare le distanze
__global__ void cudaNodeRelax(int nodo, int *graph, int *distances, int *visited, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    // Carica la riga del nodo corrente nella memoria condivisa
    extern __shared__ int sharedRow[];
    if (tid < n) {
        sharedRow[tid] = graph[nodo * n + tid];
    }
    __syncthreads();

    if (tid < n && visited[tid] == 0) { // Assicurati che il nodo non sia stato visitato
        int weight = sharedRow[tid]; // Peso dell'arco (nodo -> tid)
        if (weight != INT_MAX && weight != 0) { // Arco esistente e diverso da se stesso
            atomicMin(&distances[tid], distances[nodo] + weight); // Aggiorna la distanza con atomicMin
        }
    }
}

__global__ void findMinReductionCUDA(int *distanze, int *visitato, int n, int *minNodo, int *minDistanza) {
    // Memoria condivisa dinamica
    extern __shared__ int sharedMem[];

    // Dividi la memoria condivisa in due array distinti
    int *sharedDistanze = sharedMem;                           // Primo array: memorizza le distanze
    int *sharedNodi = &sharedDistanze[n];                     // Secondo array: memorizza gli indici dei nodi

    int tid = threadIdx.x;                   // Indice del thread locale al blocco
    int gid = blockIdx.x * blockDim.x + tid; // Indice globale del thread

    // Inizializzazione della memoria condivisa
    if (gid < n && tid < blockDim.x) {
        sharedDistanze[tid] = (visitato[gid] == 0) ? distanze[gid] : INT_MAX;
        sharedNodi[tid] = gid;
    } else {
        sharedDistanze[tid] = INT_MAX;
        sharedNodi[tid] = -1;
    }
    __syncthreads();

    // Riduzione parallela per trovare il minimo
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (gid < n && tid + stride < blockDim.x) {
            if (sharedDistanze[tid + stride] < sharedDistanze[tid]) {
                sharedDistanze[tid] = sharedDistanze[tid + stride];
                sharedNodi[tid] = sharedNodi[tid + stride];
            }
        }
        __syncthreads();
    }

    // Il thread 0 del blocco salva il risultato parziale in modo atomico
    if (tid == 0) {
        atomicExch(minNodo, sharedNodi[0]);
        atomicExch(minDistanza, sharedDistanze[0]);
        atomicExch(&visitato[sharedNodi[0]], 1); // Segna come visitato
    }
}


__global__ void initialValueKernel(int*distances, int*visited,int start,int n){
  int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        distances[id] = (id == start) ? 0 : INFINITY;
        visited[id] = 0;
    }
}

void dijkstraCUDA(int *graph, int n, int start, int *distances)  {
    // Allocazione memoria su GPU
    int *d_graph,*d_minNodo, *d_minDistanza, *d_distances, *d_visited;

    // Allocazione memoria sulla GPU
    cudaMalloc((void **)&d_graph, n * n * sizeof(int));
    cudaMalloc((void **)&d_distances, n * sizeof(int));
    cudaMalloc((void **)&d_visited, n * sizeof(int));
    cudaMalloc(&d_minNodo, sizeof(int));
    cudaMalloc(&d_minDistanza, sizeof(int));

    int blockSize = n;
    int gridDim=(n + blockSize - 1) / blockSize;
    initialValueKernel<<<gridDim,blockSize>>>(d_distances,d_visited,start,n);
    cudaDeviceSynchronize();

    // Copia il grafo e i dati iniziali sulla GPU
    cudaMemcpy(d_graph, graph, n * n * sizeof(int), cudaMemcpyHostToDevice);

    for (int i = 0; i < n; ++i) {
        // Resetta `minDistanza` al massimo valore possibile
        int h_minDistanza = INT_MAX;
        cudaMemcpy(d_minDistanza, &h_minDistanza, sizeof(int), cudaMemcpyHostToDevice);

        // Trova il nodo con la distanza minima non visitato
        blockSize = n;
        gridDim=(n + blockSize - 1) / blockSize;
        findMinReductionCUDA<<<gridDim, blockSize, blockSize * sizeof(int) * 2>>>(d_distances, d_visited, n, d_minNodo, d_minDistanza);

        if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        return;
        }

        // Recupera il nodo con distanza minima dalla GPU
        int h_minNodo;
        cudaMemcpy(&h_minNodo, d_minNodo, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(&h_minDistanza, d_minDistanza, sizeof(int), cudaMemcpyDeviceToHost);
        // Se non ci sono più nodi raggiungibili, esci
        if (h_minNodo == -1 || h_minDistanza == INT_MAX) {
            break;
        }
        cudaNodeRelax<<<gridDim, blockSize, blockSize * sizeof(int)>>>(h_minNodo, d_graph, d_distances, d_visited, n);
        cudaDeviceSynchronize();
    }

    // Copia i risultati dalla GPU alla CPU
    cudaMemcpy(distances, d_distances, n * sizeof(int), cudaMemcpyDeviceToHost);

    // Libera memoria GPU
    cudaFree(d_graph);
    cudaFree(d_distances);
    cudaFree(d_visited);
    cudaFree(d_minNodo);
    cudaFree(d_minDistanza);
}



__global__ void generateGraphKernel(int *graph, int n, int minWeight, int maxWeight, unsigned int seed) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Stato del generatore casuale
    curandState state;

    // Grid-stride loop per coprire tutta la matrice
    for (int i = idx; i < n * n; i += blockDim.x * gridDim.x) {
        int row = i / n; // Calcola la riga
        int col = i % n; // Calcola la colonna

        // Inizializza lo stato casuale
        curand_init(seed + i, 0, 0, &state);

        // Calcola il peso per la triangolare superiore
        int weight = (row < col)
                     ? ((curand(&state) % 100 < 40) ? curand(&state) % (maxWeight - minWeight + 1) + minWeight : INT_MAX)
                     : 0;

        // Scrive nella memoria globale
        if (row < col) {
            graph[row * n + col] = weight;
            graph[col * n + row] = weight; // Simmetria
        }

        // Imposta la diagonale a 0
        if (row == col) {
            graph[row * n + col] = 0;
        }
    }
}


void generateGraphCUDA(int *graph, int n, int minWeight, int maxWeight) {
    int *d_graph;
    size_t size = n * n * sizeof(int);

    // Allocazione della memoria sulla GPU
    if (cudaMalloc(&d_graph, size) != cudaSuccess) {
        std::cerr << "Errore nell'allocazione della memoria sulla GPU." << std::endl;
        return;
    }
    if (cudaMemset(d_graph, 0, size) != cudaSuccess) {
        std::cerr << "Errore durante l'inizializzazione della memoria sulla GPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Configura blocchi e griglia con valori fissi
    dim3 threadsPerBlock(1024, 1, 1);
    dim3 blocksPerGrid((n + threadsPerBlock.x - 1) / threadsPerBlock.x, 1, 1);


    // Lancia il kernel con configurazione fissa
    generateGraphKernel<<<blocksPerGrid, threadsPerBlock>>>(d_graph, n, minWeight, maxWeight, time(NULL));

    if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Copia i dati dalla GPU alla CPU
    if (cudaMemcpy(graph, d_graph, size, cudaMemcpyDeviceToHost) != cudaSuccess) {
        std::cerr << "Errore nella copia dei dati dalla GPU alla CPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Libera memoria GPU
    cudaFree(d_graph);
}


void generateGraphSerial(int *graph, int n, int minWeight, int maxWeight) {
    // Inizializza il generatore di numeri casuali
    std::srand(std::time(0));

    for (int row = 0; row < n; ++row) {
        for (int col = 0; col < n; ++col) {
            if (row == col) {
                // Imposta la diagonale principale a 0
                graph[row * n + col] = 0;
            } else if (row < col) {
                // Genera un arco casuale per la triangolare superiore
                int randomValue = std::rand() % 100;
                if (randomValue < 40) { // 40% probabilità di avere un arco
                    int weight = std::rand() % (maxWeight - minWeight + 1) + minWeight;
                    graph[row * n + col] = weight;
                    graph[col * n + row] = weight; // Simmetria
                } else {
                    graph[row * n + col] = INT_MAX; // Nessun arco
                    graph[col * n + row] = INT_MAX; // Nessun arco
                }
            }
        }
    }
}


int main() {
    int n = 32768; // Numero di nodi
    int start = 0; // Nodo iniziale
    int minWeight = 1;
    int maxWeight = 2000;

    std::vector<int> graph(n * n);
    std::vector<int> graphSerial(n * n);

    // Generazione del grafo Seriale
auto startTimeS = std::chrono::high_resolution_clock::now();
generateGraphSerial(graphSerial.data(), n, minWeight, maxWeight);
auto endTimeS = std::chrono::high_resolution_clock::now();
long serialGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(endTimeS - startTimeS).count();


// Generazione del grafo con CUDA
auto startTime = std::chrono::high_resolution_clock::now();
generateGraphCUDA(graph.data(), n, minWeight, maxWeight);
auto endTime = std::chrono::high_resolution_clock::now();
long cudaGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();


// Calcolo delle distanze con Dijkstra Seriale
std::vector<int> serialDistances(n);
startTime = std::chrono::high_resolution_clock::now();
dijkstraSerial(graph.data(), n, start, serialDistances.data());
endTime = std::chrono::high_resolution_clock::now();
long serialTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

// Calcolo delle distanze con Dijkstra CUDA
std::vector<int> cudaDistances(n);
startTime = std::chrono::high_resolution_clock::now();
dijkstraCUDA(graph.data(), n, start, cudaDistances.data());
endTime = std::chrono::high_resolution_clock::now();
long cudaTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

/*
// Stampa del grafo
std::cout << "=== Matrice del grafo ===\n";
for (int i = 0; i < n; ++i) {
    for (int j = 0; j < n; ++j) {
        if (i == j) {
            std::cout << "X "; // Diagonale principale
        } else if (graph[i * n + j] == INT_MAX) {
            std::cout << "_ "; // Nessun collegamento
        } else {
            std::cout << graph[i * n + j] << " "; // Peso dell'arco
        }
    }
    std::cout << "\n";
}

// Confronto distanze Seriali vs CUDA
std::cout << "\n=== Distanze (Seriale vs CUDA) ===\n";
std::cout << "Nodo\tSeriale\tCUDA\n";
for (int i = 0; i < n; ++i) {
    std::cout << i << "\t";
    if (serialDistances[i] == INT_MAX)
        std::cout << "∞\t";
    else
        std::cout << serialDistances[i] << "\t";

    if (cudaDistances[i] == INT_MAX)
        std::cout << "∞\n";
    else
        std::cout << cudaDistances[i] << "\n";
}
*/

// Stampa dei tempi di generazione e di esecuzione
std::cout << "\n=== Tempi di generazione ===\n";
std::cout << "Tempo di generazione Seriale: " << serialGraphTime << " µs\n";
std::cout << "Tempo di generazione CUDA: " << cudaGraphTime << " µs\n";

std::cout << "\n=== Tempi di esecuzione Dijkstra ===\n";
std::cout << "Tempo esecuzione Seriale: " << serialTime << " µs\n";
std::cout << "Tempo esecuzione CUDA: " << cudaTime << " µs\n";

return 0;
}


Overwriting dijkstra_serialMerge_3kern.cu


In [ ]:
!nvcc -rdc=true dijkstra_serialMerge_3kern.cu -lcudadevrt -o dijkstra_serialMerge_3kern.o


In [ ]:
!nvprof ./dijkstra_serialMerge_3kern.o

In [ ]:
!ncu --mode launch-and-attach -o profile --target-processes all --nvtx --call-stack --section ComputeWorkloadAnalysis --section MemoryWorkloadAnalysis -f ./dijkstra_serialMerge_3kern.o

In [ ]:
!ncu --target-processes=all ./dijkstra_serialMerge_3kern.o

==PROF== Connected to process 14359 (/content/dijkstra_serialMerge_3kern.o)
==PROF== Profiling "generateGraphKernel" - 0: 0%....50%....100% - 8 passes
==PROF== Profiling "initialValueKernel" - 1: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 2: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 3: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 4: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 5: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 6: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 7: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 8: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 9: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 10: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 11: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 12: 

In [ ]:
!./dijkstra_serialMerge_3kern.o

In [ ]:
%%writefile debug.cu
#include <iostream>
#include <vector>
#include <cstdlib>
#include <cfloat> // Per FLT_MAX
#include <cuda.h>
#include <curand_kernel.h>
#include <chrono>

// Funzione per eseguire Dijkstra in modo seriale sulla CPU
void dijkstraSerial(const float *graph, int n, int start, float *distances) {
    // Array per tenere traccia dei nodi visitati
    std::vector<int> visited(n, 0);

    // Inizializza le distanze
    for (int i = 0; i < n; ++i) {
        distances[i] = (i == start) ? 0.0f : FLT_MAX;
    }

    for (int i = 0; i < n; ++i) {
        float minDist = FLT_MAX;
        int currentNode = -1;

        // Trova il nodo con la distanza minima non ancora visitato
        for (int j = 0; j < n; ++j) {
            if (!visited[j] && distances[j] < minDist) {
                minDist = distances[j];
                currentNode = j;
            }
        }

        if (currentNode == -1) break; // Nessun altro nodo raggiungibile

        // Marca il nodo corrente come visitato
        visited[currentNode] = 1;

        // Aggiorna le distanze per i nodi adiacenti
        for (int j = 0; j < n; ++j) {
            float weight = graph[currentNode * n + j]; // Peso dell'arco currentNode -> j
            if (weight > 0.0f && weight < FLT_MAX) {  // Esiste un arco valido
                // Verifica per evitare overflow: dist[currentNode] + weight
                if (distances[currentNode] != FLT_MAX &&
                    distances[currentNode] + weight < distances[j]) {
                    distances[j] = distances[currentNode] + weight;
                }
            }
        }
    }
}
// Kernel CUDA per aggiornare le distanze
__global__ void cudaNodeRelax(int nodo, float *graph, float *distances, int *visited, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    // Carica la riga del nodo corrente nella memoria condivisa
    extern __shared__ float sharedRow[];
    if (tid < n) {
        sharedRow[tid] = graph[nodo * n + tid];
    }
    __syncthreads();

    if (tid < n && visited[tid] == 0) { // Assicurati che il nodo non sia stato visitato
        float weight = sharedRow[tid]; // Peso dell'arco (nodo -> tid)
        if (weight < FLT_MAX && weight > 0.0f) { // Arco esistente e diverso da se stesso
            atomicMin((int *)&distances[tid], __float_as_int(distances[nodo] + weight)); // Aggiorna la distanza con atomicMin
        }
    }
}

__global__ void findMinReductionCUDA(float *distanze, int *visitato, int n, int *minNodo, float *minDistanza) {
    // Memoria condivisa dinamica
    extern __shared__ float sharedMem[];

    // Dividi la memoria condivisa in due array distinti
    float *sharedDistanze = sharedMem;                        // Primo array: memorizza le distanze
    int *sharedNodi = (int *)&sharedDistanze[n];              // Secondo array: memorizza gli indici dei nodi

    int tid = threadIdx.x;                   // Indice del thread locale al blocco
    int gid = blockIdx.x * blockDim.x + tid; // Indice globale del thread

    // Inizializzazione della memoria condivisa
    if (gid < n && tid < blockDim.x) {
        sharedDistanze[tid] = (visitato[gid] == 0) ? distanze[gid] : FLT_MAX;
        sharedNodi[tid] = gid;
    } else {
        sharedDistanze[tid] = FLT_MAX;
        sharedNodi[tid] = -1;
    }
    __syncthreads();

    // Riduzione parallela per trovare il minimo
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (tid + stride < blockDim.x && sharedDistanze[tid + stride] < sharedDistanze[tid]) {
            sharedDistanze[tid] = sharedDistanze[tid + stride];
            sharedNodi[tid] = sharedNodi[tid + stride];
        }
        __syncthreads();
    }

    // Il thread 0 del blocco salva il risultato parziale in modo atomico
    if (tid == 0) {
        atomicExch(minNodo, sharedNodi[0]);
        atomicExch((int *)minDistanza, __float_as_int(sharedDistanze[0]));
        atomicExch(&visitato[sharedNodi[0]], 1); // Segna come visitato
    }
}
__global__ void initialValueKernel(float *distances, int *visited, int start, int n) {
    int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        distances[id] = (id == start) ? 0.0f : FLT_MAX;
        visited[id] = 0;
    }
}

void dijkstraCUDA(float *graph, int n, int start, float *distances) {
    // Allocazione memoria su GPU
    float *d_graph, *d_minDistanza, *d_distances;
    int *d_minNodo, *d_visited;

    // Allocazione memoria sulla GPU
    cudaMalloc((void **)&d_graph, n * n * sizeof(float));
    cudaMalloc((void **)&d_distances, n * sizeof(float));
    cudaMalloc((void **)&d_visited, n * sizeof(int));
    cudaMalloc((void **)&d_minNodo, sizeof(int));
    cudaMalloc((void **)&d_minDistanza, sizeof(float));

    int blockSize = n;
    int gridDim = (n + blockSize - 1) / blockSize;

    // Inizializza distanze e visitati
    initialValueKernel<<<gridDim, blockSize>>>(d_distances, d_visited, start, n);
    cudaDeviceSynchronize();

    // Copia il grafo sulla GPU
    cudaMemcpy(d_graph, graph, n * n * sizeof(float), cudaMemcpyHostToDevice);

    for (int i = 0; i < n; ++i) {
        // Resetta `minDistanza` al massimo valore possibile
        float h_minDistanza = FLT_MAX;
        cudaMemcpy(d_minDistanza, &h_minDistanza, sizeof(float), cudaMemcpyHostToDevice);

        // Trova il nodo con la distanza minima non visitato
        blockSize = n;
        gridDim = (n + blockSize - 1) / blockSize;
        findMinReductionCUDA<<<gridDim, blockSize, blockSize * (sizeof(float) + sizeof(int))>>>(
            d_distances, d_visited, n, d_minNodo, d_minDistanza);

        if (cudaDeviceSynchronize() != cudaSuccess) {
            std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
            return;
        }

        // Recupera il nodo con distanza minima dalla GPU
        int h_minNodo;
        cudaMemcpy(&h_minNodo, d_minNodo, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(&h_minDistanza, d_minDistanza, sizeof(float), cudaMemcpyDeviceToHost);

        // Se non ci sono più nodi raggiungibili, esci
        if (h_minNodo == -1 || h_minDistanza == FLT_MAX) {
            break;
        }

        // Aggiorna le distanze per il nodo corrente
        cudaNodeRelax<<<gridDim, blockSize, blockSize * sizeof(float)>>>(h_minNodo, d_graph, d_distances, d_visited, n);
        cudaDeviceSynchronize();
    }

    // Copia i risultati dalla GPU alla CPU
    cudaMemcpy(distances, d_distances, n * sizeof(float), cudaMemcpyDeviceToHost);

    // Libera memoria GPU
    cudaFree(d_graph);
    cudaFree(d_distances);
    cudaFree(d_visited);
    cudaFree(d_minNodo);
    cudaFree(d_minDistanza);
}
__global__ void generateGraphKernel(float *graph, int n, float minWeight, float maxWeight, unsigned int seed) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Stato del generatore casuale
    curandState state;

    // Grid-stride loop per coprire tutta la matrice
    for (int i = idx; i < n * n; i += blockDim.x * gridDim.x) {
        int row = i / n; // Calcola la riga
        int col = i % n; // Calcola la colonna

        // Inizializza lo stato casuale
        curand_init(seed + i, 0, 0, &state);

        // Calcola il peso per la triangolare superiore
        if (row < col) {
            float randomProb = curand_uniform(&state);
            float weight = (randomProb < 0.4f)
                           ? curand_uniform(&state) * (maxWeight - minWeight) + minWeight
                           : FLT_MAX;

            // Scrive nella memoria globale
            graph[row * n + col] = weight;
            graph[col * n + row] = weight; // Simmetria
        } else if (row == col) {
            // Imposta la diagonale a 0
            graph[row * n + col] = 0.0f;
        }
    }
}

void generateGraphCUDA(float *graph, int n, float minWeight, float maxWeight) {
    float *d_graph;
    size_t size = n * n * sizeof(float);

    // Allocazione della memoria sulla GPU
    if (cudaMalloc(&d_graph, size) != cudaSuccess) {
        std::cerr << "Errore nell'allocazione della memoria sulla GPU." << std::endl;
        return;
    }

    if (cudaMemset(d_graph, 0, size) != cudaSuccess) {
        std::cerr << "Errore durante l'inizializzazione della memoria sulla GPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Configura blocchi e griglia
    int threadsPerBlock = 1024;
    int blocksPerGrid = (n * n + threadsPerBlock - 1) / threadsPerBlock;

    // Lancia il kernel
    generateGraphKernel<<<blocksPerGrid, threadsPerBlock>>>(d_graph, n, minWeight, maxWeight, time(NULL));

    if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Copia i dati dalla GPU alla CPU
    if (cudaMemcpy(graph, d_graph, size, cudaMemcpyDeviceToHost) != cudaSuccess) {
        std::cerr << "Errore nella copia dei dati dalla GPU alla CPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Libera memoria GPU
    cudaFree(d_graph);
}

void generateGraphSerial(float *graph, int n, float minWeight, float maxWeight) {
    // Inizializza il generatore di numeri casuali
    std::srand(std::time(0));

    for (int row = 0; row < n; ++row) {
        for (int col = 0; col < n; ++col) {
            if (row == col) {
                // Imposta la diagonale principale a 0
                graph[row * n + col] = 0.0f;
            } else if (row < col) {
                // Genera un arco casuale per la triangolare superiore
                int randomValue = std::rand() % 100;
                if (randomValue < 40) { // 40% probabilità di avere un arco
                    float weight = static_cast<float>(std::rand()) / RAND_MAX * (maxWeight - minWeight) + minWeight;
                    graph[row * n + col] = weight;
                    graph[col * n + row] = weight; // Simmetria
                } else {
                    graph[row * n + col] = FLT_MAX; // Nessun arco
                    graph[col * n + row] = FLT_MAX; // Nessun arco
                }
            }
        }
    }
}

int main() {
    int n = 4096; // Numero di nodi
    int start = 0; // Nodo iniziale
    float minWeight = 1.0f;
    float maxWeight = 2000.0f;

    std::vector<float> graph(n * n);
    std::vector<float> graphSerial(n * n);

     // Generazione del grafo Seriale
    auto startTimeS = std::chrono::high_resolution_clock::now();
    generateGraphSerial(graphSerial.data(), n, minWeight, maxWeight);
    auto endTimeS = std::chrono::high_resolution_clock::now();
    long serialGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(endTimeS - startTimeS).count();


    // Generazione del grafo con CUDA
    auto startTime = std::chrono::high_resolution_clock::now();
    generateGraphCUDA(graph.data(), n, minWeight, maxWeight);
    auto endTime = std::chrono::high_resolution_clock::now();
    long cudaGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

    // Calcolo delle distanze con Dijkstra Seriale
    std::vector<float> serialDistances(n);
    startTime = std::chrono::high_resolution_clock::now();
    dijkstraSerial(graph.data(), n, start, serialDistances.data());
    endTime = std::chrono::high_resolution_clock::now();
    long serialTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

    // Calcolo delle distanze con Dijkstra CUDA
    std::vector<float> cudaDistances(n);
    startTime = std::chrono::high_resolution_clock::now();
    dijkstraCUDA(graph.data(), n, start, cudaDistances.data());
    endTime = std::chrono::high_resolution_clock::now();
    long cudaTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

  /*
    // Stampa del grafo
    std::cout << "=== Matrice del grafo ===\n";
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            if (i == j) {
                std::cout << "X "; // Diagonale principale
            } else if (graph[i * n + j] == FLT_MAX) {
                std::cout << "_ "; // Nessun collegamento
            } else {
                std::cout << graph[i * n + j] << " "; // Peso dell'arco
            }
        }
        std::cout << "\n";
    }

    // Confronto distanze Seriali vs CUDA
    std::cout << "\n=== Distanze (Seriale vs CUDA) ===\n";
    std::cout << "Nodo\tSeriale\tCUDA\n";
    for (int i = 0; i < n; ++i) {
        std::cout << i << "\t";
        if (serialDistances[i] == FLT_MAX)
            std::cout << "∞\t";
        else
            std::cout << serialDistances[i] << "\t";

        if (cudaDistances[i] == FLT_MAX)
            std::cout << "∞\n";
        else
            std::cout << cudaDistances[i] << "\n";
    }
  */

    // Stampa dei tempi di generazione e di esecuzione
    std::cout << "\n=== Tempi di generazione ===\n";
    std::cout << "Tempo di generazione Seriale: " << serialGraphTime << " µs\n";
    std::cout << "Tempo di generazione CUDA: " << cudaGraphTime << " µs\n";

    std::cout << "\n=== Tempi di esecuzione Dijkstra ===\n";
    std::cout << "Tempo esecuzione Seriale: " << serialTime << " µs\n";
    std::cout << "Tempo esecuzione CUDA: " << cudaTime << " µs\n";

    return 0;
}


Overwriting debug.cu


In [ ]:
!nvcc -G -g debug.cu -o debug.o
!./debug.o

In [ ]:
!ncu --set full -o test_rep3 ./debug.o

In [ ]:
%%writefile debug_double.cu
#include <iostream>
#include <vector>
#include <cstdlib>
#include <cfloat> // Per DBL_MAX
#include <cuda.h>
#include <curand_kernel.h>
#include <chrono>

// Funzione per eseguire Dijkstra in modo seriale sulla CPU
void dijkstraSerial(const double *graph, int n, int start, double *distances) {
    // Array per tenere traccia dei nodi visitati
    std::vector<int> visited(n, 0);

    // Inizializza le distanze
    for (int i = 0; i < n; ++i) {
        distances[i] = (i == start) ? 0.0 : DBL_MAX;
    }

    for (int i = 0; i < n; ++i) {
        double minDist = DBL_MAX;
        int currentNode = -1;

        // Trova il nodo con la distanza minima non ancora visitato
        for (int j = 0; j < n; ++j) {
            if (!visited[j] && distances[j] < minDist) {
                minDist = distances[j];
                currentNode = j;
            }
        }

        if (currentNode == -1) break; // Nessun altro nodo raggiungibile

        // Marca il nodo corrente come visitato
        visited[currentNode] = 1;

        // Aggiorna le distanze per i nodi adiacenti
        for (int j = 0; j < n; ++j) {
            double weight = graph[currentNode * n + j]; // Peso dell'arco currentNode -> j
            if (weight > 0.0 && weight < DBL_MAX) {  // Esiste un arco valido
                // Verifica per evitare overflow: dist[currentNode] + weight
                if (distances[currentNode] != DBL_MAX &&
                    distances[currentNode] + weight < distances[j]) {
                    distances[j] = distances[currentNode] + weight;
                }
            }
        }
    }
}

// Kernel CUDA per aggiornare le distanze
__global__ void cudaNodeRelax(int nodo, double *graph, double *distances, int *visited, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    // Carica la riga del nodo corrente nella memoria condivisa
    extern __shared__ double sharedRow[];
    if (tid < n) {
        sharedRow[tid] = graph[nodo * n + tid];
    }
    __syncthreads();

    if (tid < n && visited[tid] == 0) { // Assicurati che il nodo non sia stato visitato
        double weight = sharedRow[tid]; // Peso dell'arco (nodo -> tid)
        if (weight < DBL_MAX && weight > 0.0) { // Arco esistente e diverso da se stesso
            atomicMin((long long *)&distances[tid], __double_as_longlong(distances[nodo] + weight)); // Aggiorna la distanza con atomicMin
        }
    }
}

__global__ void findMinReductionCUDA(double *distanze, int *visitato, int n, int *minNodo, double *minDistanza) {
    // Memoria condivisa dinamica
    extern __shared__ double sharedMem[];

    // Dividi la memoria condivisa in due array distinti
    double *sharedDistanze = sharedMem;                        // Primo array: memorizza le distanze
    int *sharedNodi = (int *)&sharedDistanze[n];               // Secondo array: memorizza gli indici dei nodi

    int tid = threadIdx.x;                   // Indice del thread locale al blocco
    int gid = blockIdx.x * blockDim.x + tid; // Indice globale del thread

    // Inizializzazione della memoria condivisa
    if (gid < n && tid < blockDim.x) {
        sharedDistanze[tid] = (visitato[gid] == 0) ? distanze[gid] : DBL_MAX;
        sharedNodi[tid] = gid;
    } else {
        sharedDistanze[tid] = DBL_MAX;
        sharedNodi[tid] = -1;
    }
    __syncthreads();

    // Riduzione parallela per trovare il minimo
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (tid + stride < blockDim.x && sharedDistanze[tid + stride] < sharedDistanze[tid]) {
            sharedDistanze[tid] = sharedDistanze[tid + stride];
            sharedNodi[tid] = sharedNodi[tid + stride];
        }
        __syncthreads();
    }

    // Il thread 0 del blocco salva il risultato parziale in modo atomico
    if (tid == 0) {
        atomicExch(minNodo, sharedNodi[0]);
        atomicExch((long long *)minDistanza, __double_as_longlong(sharedDistanze[0]));
        atomicExch(&visitato[sharedNodi[0]], 1); // Segna come visitato
    }
}

__global__ void initialValueKernel(double *distances, int *visited, int start, int n) {
    int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        distances[id] = (id == start) ? 0.0 : DBL_MAX;
        visited[id] = 0;
    }
}
void dijkstraCUDA(double *graph, int n, int start, double *distances) {
    // Allocazione memoria su GPU
    double *d_graph, *d_minDistanza, *d_distances;
    int *d_minNodo, *d_visited;

    // Allocazione memoria sulla GPU
    cudaMalloc((void **)&d_graph, n * n * sizeof(double));
    cudaMalloc((void **)&d_distances, n * sizeof(double));
    cudaMalloc((void **)&d_visited, n * sizeof(int));
    cudaMalloc((void **)&d_minNodo, sizeof(int));
    cudaMalloc((void **)&d_minDistanza, sizeof(double));

    int blockSize = n;
    int gridDim = (n + blockSize - 1) / blockSize;

    // Inizializza distanze e visitati
    initialValueKernel<<<gridDim, blockSize>>>(d_distances, d_visited, start, n);
    cudaDeviceSynchronize();

    // Copia il grafo sulla GPU
    cudaMemcpy(d_graph, graph, n * n * sizeof(double), cudaMemcpyHostToDevice);

    for (int i = 0; i < n; ++i) {
        // Resetta `minDistanza` al massimo valore possibile
        double h_minDistanza = DBL_MAX;
        cudaMemcpy(d_minDistanza, &h_minDistanza, sizeof(double), cudaMemcpyHostToDevice);

        // Trova il nodo con la distanza minima non visitato
        blockSize = n;
        gridDim = (n + blockSize - 1) / blockSize;
        findMinReductionCUDA<<<gridDim, blockSize, blockSize * (sizeof(double) + sizeof(int))>>>(
            d_distances, d_visited, n, d_minNodo, d_minDistanza);

        if (cudaDeviceSynchronize() != cudaSuccess) {
            std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
            return;
        }

        // Recupera il nodo con distanza minima dalla GPU
        int h_minNodo;
        cudaMemcpy(&h_minNodo, d_minNodo, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(&h_minDistanza, d_minDistanza, sizeof(double), cudaMemcpyDeviceToHost);

        // Se non ci sono più nodi raggiungibili, esci
        if (h_minNodo == -1 || h_minDistanza == DBL_MAX) {
            break;
        }

        // Aggiorna le distanze per il nodo corrente
        cudaNodeRelax<<<gridDim, blockSize, blockSize * sizeof(double)>>>(h_minNodo, d_graph, d_distances, d_visited, n);
        cudaDeviceSynchronize();
    }

    // Copia i risultati dalla GPU alla CPU
    cudaMemcpy(distances, d_distances, n * sizeof(double), cudaMemcpyDeviceToHost);

    // Libera memoria GPU
    cudaFree(d_graph);
    cudaFree(d_distances);
    cudaFree(d_visited);
    cudaFree(d_minNodo);
    cudaFree(d_minDistanza);
}

__global__ void generateGraphKernel(double *graph, int n, double minWeight, double maxWeight, unsigned int seed) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Stato del generatore casuale
    curandState state;

    // Grid-stride loop per coprire tutta la matrice
    for (int i = idx; i < n * n; i += blockDim.x * gridDim.x) {
        int row = i / n; // Calcola la riga
        int col = i % n; // Calcola la colonna

        // Inizializza lo stato casuale
        curand_init(seed + i, 0, 0, &state);

        // Calcola il peso per la triangolare superiore
        if (row < col) {
            double randomProb = curand_uniform_double(&state);
            double weight = (randomProb < 0.4)
                           ? curand_uniform_double(&state) * (maxWeight - minWeight) + minWeight
                           : DBL_MAX;

            // Scrive nella memoria globale
            graph[row * n + col] = weight;
            graph[col * n + row] = weight; // Simmetria
        } else if (row == col) {
            // Imposta la diagonale a 0
            graph[row * n + col] = 0.0;
        }
    }
}
void generateGraphCUDA(double *graph, int n, double minWeight, double maxWeight) {
    double *d_graph;
    size_t size = n * n * sizeof(double);

    // Allocazione della memoria sulla GPU
    if (cudaMalloc(&d_graph, size) != cudaSuccess) {
        std::cerr << "Errore nell'allocazione della memoria sulla GPU." << std::endl;
        return;
    }

    if (cudaMemset(d_graph, 0, size) != cudaSuccess) {
        std::cerr << "Errore durante l'inizializzazione della memoria sulla GPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Configura blocchi e griglia
    int threadsPerBlock = 1024;
    int blocksPerGrid = (n * n + threadsPerBlock - 1) / threadsPerBlock;

    // Lancia il kernel
    generateGraphKernel<<<blocksPerGrid, threadsPerBlock>>>(d_graph, n, minWeight, maxWeight, time(NULL));

    if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Copia i dati dalla GPU alla CPU
    if (cudaMemcpy(graph, d_graph, size, cudaMemcpyDeviceToHost) != cudaSuccess) {
        std::cerr << "Errore nella copia dei dati dalla GPU alla CPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Libera memoria GPU
    cudaFree(d_graph);
}

void generateGraphSerial(double *graph, int n, double minWeight, double maxWeight) {
    // Inizializza il generatore di numeri casuali
    std::srand(std::time(0));

    for (int row = 0; row < n; ++row) {
        for (int col = 0; col < n; ++col) {
            if (row == col) {
                // Imposta la diagonale principale a 0
                graph[row * n + col] = 0.0;
            } else if (row < col) {
                // Genera un arco casuale per la triangolare superiore
                int randomValue = std::rand() % 100;
                if (randomValue < 40) { // 40% probabilità di avere un arco
                    double weight = static_cast<double>(std::rand()) / RAND_MAX * (maxWeight - minWeight) + minWeight;
                    graph[row * n + col] = weight;
                    graph[col * n + row] = weight; // Simmetria
                } else {
                    graph[row * n + col] = DBL_MAX; // Nessun arco
                    graph[col * n + row] = DBL_MAX; // Nessun arco
                }
            }
        }
    }
}

int main() {
    int n = 4096; // Numero di nodi
    int start = 0; // Nodo iniziale
    double minWeight = 1.0;
    double maxWeight = 2000.0;

    std::vector<double> graph(n * n);
    std::vector<double> graphSerial(n * n);

    // Generazione del grafo Seriale
    auto startTimeS = std::chrono::high_resolution_clock::now();
    generateGraphSerial(graphSerial.data(), n, minWeight, maxWeight);
    auto endTimeS = std::chrono::high_resolution_clock::now();
    long serialGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(endTimeS - startTimeS).count();

    // Generazione del grafo con CUDA
    auto startTime = std::chrono::high_resolution_clock::now();
    generateGraphCUDA(graph.data(), n, minWeight, maxWeight);
    auto endTime = std::chrono::high_resolution_clock::now();
    long cudaGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

    // Calcolo delle distanze con Dijkstra Seriale
    std::vector<double> serialDistances(n);
    startTime = std::chrono::high_resolution_clock::now();
    dijkstraSerial(graph.data(), n, start, serialDistances.data());
    endTime = std::chrono::high_resolution_clock::now();
    long serialTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

    // Calcolo delle distanze con Dijkstra CUDA
    std::vector<double> cudaDistances(n);
    startTime = std::chrono::high_resolution_clock::now();
    dijkstraCUDA(graph.data(), n, start, cudaDistances.data());
    endTime = std::chrono::high_resolution_clock::now();
    long cudaTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

    // Stampa dei tempi di generazione e di esecuzione
    std::cout << "\n=== Tempi di generazione ===\n";
    std::cout << "Tempo di generazione Seriale: " << serialGraphTime << " µs\n";
    std::cout << "Tempo di generazione CUDA: " << cudaGraphTime << " µs\n";

    std::cout << "\n=== Tempi di esecuzione Dijkstra ===\n";
    std::cout << "Tempo esecuzione Seriale: " << serialTime << " µs\n";
    std::cout << "Tempo esecuzione CUDA: " << cudaTime << " µs\n";

    return 0;
}


In [ ]:
!nvcc -G -g debug_double.cu -o debug_double.o
!./debug_double.o

In [ ]:
!ncu --set full -o test_rep3 ./debug_double.o